In [1]:
!wget -O Audio_Song_Actors_01-24.zip "https://zenodo.org/records/1188976/files/Audio_Song_Actors_01-24.zip?download=1"
!unzip Audio_Song_Actors_01-24.zip -d Audio_Song_Actors
!wget -O Audio_Speech_Actors_01-24.zip "https://zenodo.org/records/1188976/files/Audio_Speech_Actors_01-24.zip?download=1"
!unzip Audio_Speech_Actors_01-24.zip -d Audio_Speech_Actors

--2025-06-13 11:25:48--  https://zenodo.org/records/1188976/files/Audio_Song_Actors_01-24.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225505317 (215M) [application/octet-stream]
Saving to: ‘Audio_Song_Actors_01-24.zip’

Audio_Song_Actors_0 100%[===================>] 215.06M   109MB/s    in 2.0s    

2025-06-13 11:25:50 (109 MB/s) - ‘Audio_Song_Actors_01-24.zip’ saved [225505317/225505317]

Archive:  Audio_Song_Actors_01-24.zip
   creating: Audio_Song_Actors/Actor_01/
  inflating: Audio_Song_Actors/Actor_01/03-02-01-01-01-01-01.wav  
  inflating: Audio_Song_Actors/Actor_01/03-02-01-01-01-02-01.wav  
  inflating: Audio_Song_Actors/Actor_01/03-02-01-01-02-01-01.wav  
  inflating: Audio_Song_Actors/Actor_01/03-02-01-01-02-02-01.wav  
  inflating: Audio_Song_Actors/Actor_01/03-02-02-01-01-01-01.wav  
  inflati

In [2]:
!pip install transformers datasets torchaudio evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00


In [5]:
import os

root_dir = "/content/audio_data"
all_audio_paths = []

# Traverse recursively and collect all .wav files
for folder, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".wav"):
            all_audio_paths.append(os.path.join(folder, file))

print(f"Total WAV files found: {len(all_audio_paths)}")


Total WAV files found: 2452


In [6]:
from sklearn.model_selection import train_test_split

train_files, val_files = train_test_split(all_audio_paths, test_size=0.2, random_state=42)


In [7]:
import torchaudio
import torch
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

extractor = AutoFeatureExtractor.from_pretrained("Hatman/audio-emotion-detection")
model = AutoModelForAudioClassification.from_pretrained("Hatman/audio-emotion-detection")
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

id2label = model.config.id2label


preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.65k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [8]:
results = []

for path in val_files:
    waveform, sr = torchaudio.load(path)
    if sr != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(waveform)
    waveform = waveform.squeeze()

    inputs = extractor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(logits, dim=-1).item()
        label = id2label[pred]

    results.append({"file": path, "predicted_emotion": label})

In [9]:
import pandas as pd
df = pd.DataFrame(results)
df.to_csv("emotion_predictions.csv", index=False)
print(df.head())

                                                file predicted_emotion
0  /content/audio_data/Audio_Song_Actors/Actor_11...             Happy
1  /content/audio_data/Audio_Speech_Actors/Actor_...             Angry
2  /content/audio_data/Audio_Speech_Actors/Actor_...             Happy
3  /content/audio_data/Audio_Song_Actors/Actor_08...             Happy
4  /content/audio_data/Audio_Song_Actors/Actor_21...             Angry
